<a href="https://colab.research.google.com/github/singhayushh/EC881--Assignment/blob/linux/DiabeticRetinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

The goal is to make a highly accurate diabetic retinopathy model by using different CNN architectures (viz. MobileNet, EfficientNet, Inception V3 and ResNet) for a comparative study and push the best working model to deployment for live usage of the AI for actual patient images in various medicinal institutes.

The model can be massively improved with:

- high-resolution images
- better data sampling
- ensuring there is no leaking between training and validation sets.
- better target variable (age) normalization
- pretrained models
- attention/related techniques to focus on areas

************

### Authors

- [Ayush Singh](https://github.com/singhayushh)
- [Agni Sain](https://linkedin.com/in/)
- [Mayukh Sen](https://linkedin.com/in/)
- [Aryan Shaw](https://linkedin.com/in/)

************

### The Model

The model we create will be run through four different CNN architectures:
- MobileNet v3
- Inception v3
- ResNet
- EfficientNet
- DenseNet

All five models will be trained and tested on the same data and based on the output, the most accurate model will be used in the live production environment.


## 1. Download Data

#### 1.1. Mount google drive

We will be using the test.001 and train.001 images from the Kaggle 2015 dataset, which have been uploaded to google drive for easier access.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### 1.2. Unzip dataset

In this step, we will unzip the dataset to train and test directories.

In [ ]:
# download dataset from kaggle cli
!kaggle competitions download -c 'diabetic-retinopathy-detection'

In [ ]:
# Create train and test directories
!mkdir train
!mkdir test

# Move ZIP files to their directories
!mv /content/drive/MyDrive/dataset/test.* test
!mv /content/drive/MyDrive/dataset/train* train

# Extract data
!7za x train/train.zip
!7za x test/test.zip

!rm train/train.zip
!rm test/test.zip

## 2. Image Preprocessing

#### 2.1. Crop and Resize

All images were scaled down to 256 by 256. Despite taking longer to train, the detail present in photos of this size is much greater then at 128 by 128.

In [ ]:
# package imports
import os
import sys
from PIL import ImageFile
from skimage import io
from skimage.transform import resize
import numpy as np

ImageFile.LOAD_TRUNCATED_IMAGES = True

# utility function to create directory with given name if absent
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# crop and resize given image and save to new path
def crop_and_resize_images(path, new_path, cropx, cropy, img_size=256):
    create_directory(new_path)
    dirs = [l for l in os.listdir(path) if l != '.DS_Store']
    total = 0
    for item in dirs:
        img = io.imread(path+item)
        y,x,channel = img.shape
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)
        img = img[starty:starty+cropy,startx:startx+cropx]
        img = resize(img, (256,256))
        io.imsave(str(new_path + item), img)
        total += 1
        print("Saving: ", item, total)


if __name__ == '__main__':
    crop_and_resize_images(path='/content/train/', new_path='/content/train-resized-256/', cropx=1800, cropy=1800, img_size=256)
    crop_and_resize_images(path='/content/test/', new_path='/content/test-resized-256/', cropx=1800, cropy=1800, img_size=256)

#### 2.2. Training data pruning

Scikit-Image raised multiple warnings during resizing, due to these images having no color space. Because of this, any images that were completely black were removed from the training data.

In [ ]:
# package imports
import time
import numpy as np
import pandas as pd
from PIL import Image

# create a 'image' named column with non-black images
def find_black_images(file_path, df):
    lst_imgs = [l for l in df['image']]
    return [1 if np.mean(np.array(Image.open(file_path + img))) == 0 else 0 for img in lst_imgs]

if __name__ == '__main__':
    start_time = time.time()
    trainLabels = pd.read_csv('/content/labels/trainLabels.csv')

    trainLabels['image'] = [i + '.jpeg' for i in trainLabels['image']]
    trainLabels['black'] = np.nan

    trainLabels['black'] = find_black_images('/content/train-resized-256/', trainLabels)
    trainLabels = trainLabels.loc[trainLabels['black'] == 0]
    trainLabels.to_csv('trainLabels_master.csv', index=False, header=True)

    print("Completed")
    print("--- %s seconds ---" % (time.time() - start_time))

#### 2.3. Image Rotation

In order to reduce noise from the images, all images were rotated and mirrored.

In [ ]:
# packages import
import pandas as pd
import numpy as np
from skimage import io
from skimage.transform import rotate
from cv2 import cv2
import os
import time

def rotate_images(file_path, degrees_of_rotation, lst_imgs):
    for l in lst_imgs:
        img = io.imread(file_path + str(l) + '.jpeg')
        img = rotate(img, degrees_of_rotation)
        io.imsave(file_path + str(l) + '_' + str(degrees_of_rotation) + '.jpeg', img)


def mirror_images(file_path, mirror_direction, lst_imgs):
    for l in lst_imgs:
        img = cv2.imread(file_path + str(l) + '.jpeg')
        img = cv2.flip(img, 1)
        cv2.imwrite(file_path + str(l) + '_mir' + '.jpeg', img)

if __name__ == '__main__':
    start_time = time.time()
    trainLabels = pd.read_csv("/content/labels/trainLabels_master.csv")

    trainLabels['image'] = trainLabels['image'].str.rstrip('.jpeg')
    trainLabels_no_DR = trainLabels[trainLabels['level'] == 0]
    trainLabels_DR = trainLabels[trainLabels['level'] >= 1]

    lst_imgs_no_DR = [i for i in trainLabels_no_DR['image']]
    lst_imgs_DR = [i for i in trainLabels_DR['image']]

    # Mirror Images with no DR one time
    print("Mirroring Non-DR Images")
    mirror_images('/content/train-resized-256/', 1, lst_imgs_no_DR)


    # Rotate all images that have any level of DR
    print("Rotating 90 Degrees")
    rotate_images('/content/train-resized-256/', 90, lst_imgs_DR)
    print("Rotating 120 Degrees")
    rotate_images('/content/train-resized-256/', 120, lst_imgs_DR)
    print("Rotating 180 Degrees")
    rotate_images('/content/train-resized-256/', 180, lst_imgs_DR)
    print("Rotating 270 Degrees")
    rotate_images('/content/train-resized-256/', 270, lst_imgs_DR)
    print("Mirroring DR Images")
    mirror_images('/content/train-resized-256/', 0, lst_imgs_DR)
    print("Completed")
    print("--- %s seconds ---" % (time.time() - start_time))

#### 2.4. Reconciling image labels

In this step, we read all image files into a list, remove their suffixes and write the resultant list into a new csv.

In [ ]:
import os
import pandas as pd

def get_lst_images(file_path):
    return [i for i in os.listdir(file_path) if i != '.DS_Store']

if __name__ == '__main__':
    trainLabels = pd.read_csv("../labels/trainLabels_master.csv")
    lst_imgs = get_lst_images('../data/train-resized-256/')
    new_trainLabels = pd.DataFrame({'image': lst_imgs})
    new_trainLabels['image2'] = new_trainLabels.image
    new_trainLabels['image2'] = new_trainLabels.loc[:, 'image2'].apply(lambda x: '_'.join(x.split('_')[0:2]))
    new_trainLabels['image2'] = new_trainLabels.loc[:, 'image2'].apply(
        lambda x: '_'.join(x.split('_')[0:2]).strip('.jpeg') + '.jpeg')
    new_trainLabels.columns = ['train_image_name', 'image']
    trainLabels = pd.merge(trainLabels, new_trainLabels, how='outer', on='image')
    trainLabels.drop(['black'], axis=1, inplace=True)
    trainLabels = trainLabels.dropna()
    print(trainLabels.shape)
    print("Writing CSV")
    trainLabels.to_csv('../labels/trainLabels_master_256_v2.csv', index=False, header=True)

#### 2.5. Saving images as numpy array

In this step, 
- we first append the suffix ".jpeg" for all images in the dataframe
- save the data object as a numpy file, used for saving train and test arrays

In [ ]:
import time
import numpy as np
import pandas as pd
from PIL import Image


def change_image_name(df, column):
    return [i + '.jpeg' for i in df[column]]


def convert_images_to_arrays_train(file_path, df):
    lst_imgs = [l for l in df['train_image_name']]
    return np.array([np.array(Image.open(file_path + img)) for img in lst_imgs])


def save_to_array(arr_name, arr_object):
    return np.save(arr_name, arr_object)


if __name__ == '__main__':
    start_time = time.time()
    labels = pd.read_csv("../labels/trainLabels_master_256_v2.csv")
    print("Writing Train Array")
    X_train = convert_images_to_arrays_train('../data/train-resized-256/', labels)
    print(X_train.shape)
    print("Saving Train Array")
    save_to_array('../data/X_train.npy', X_train)
    print("--- %s seconds ---" % (time.time() - start_time))

#### Convolutional Neural Network

Functions involed:
- Splitting the data into test and training datasets (X_train, X_test, y_train, y_test)
- Reshaping the data into the format for CNN
- Defining the CNN model using Sequential
- Saving the model as a .h5 file to be able to use it for live prediction in the web

In [ ]:
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

np.random.seed(1337)


def split_data(X, y, test_data_size):
    return train_test_split(X, y, test_size=test_data_size, random_state=42)


def reshape_data(arr, img_rows, img_cols, channels):
    return arr.reshape(arr.shape[0], img_rows, img_cols, channels)


def cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus):
    model = Sequential()
    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=(img_rows, img_cols, channels), activation="relu"))
    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))
    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    print("Model flattened out to: ", model.output_shape)
    model.add(Dense(128))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model = multi_gpu_model(model, gpus=nb_gpus)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    stop = EarlyStopping(monitor='val_acc',
                         min_delta=0.001,
                         patience=2,
                         verbose=0,
                         mode='auto')
    tensor_board = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_split=0.2,
              class_weight='auto',
              callbacks=[stop, tensor_board])
    return model


def save_model(model, score, model_name):
    if score >= 0.75:
        print("Saving Model")
        model.save("../models/" + model_name + "_recall_" + str(round(score, 4)) + ".h5")
    else:
        print("Model Not Saved.  Score: ", score)


if __name__ == '__main__':
    # Specify parameters before model is run.
    batch_size = 512
    nb_classes = 2
    nb_epoch = 30

    img_rows, img_cols = 256, 256
    channels = 3
    nb_filters = 32
    kernel_size = (8, 8)

    # Import data
    labels = pd.read_csv("../labels/trainLabels_master_256_v2.csv")
    X = np.load("../data/X_train_256_v2.npy")
    y = np.array([1 if l >= 1 else 0 for l in labels['level']])

    print("Splitting data into test/ train datasets")
    X_train, X_test, y_train, y_test = split_data(X, y, 0.2)

    print("Reshaping Data")
    X_train = reshape_data(X_train, img_rows, img_cols, channels)
    X_test = reshape_data(X_test, img_rows, img_cols, channels)

    print("X_train Shape: ", X_train.shape)
    print("X_test Shape: ", X_test.shape)

    input_shape = (img_rows, img_cols, channels)

    print("Normalizing Data")
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    X_train /= 255
    X_test /= 255

    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    print("y_train Shape: ", y_train.shape)
    print("y_test Shape: ", y_test.shape)

    print("Training Model")

    model = cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size,
                      nb_classes, nb_gpus=8)

    print("Predicting")
    y_pred = model.predict(X_test)

    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

    y_test = np.argmax(y_test, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("Precision: ", precision)
    print("Recall: ", recall)

    save_model(model=model, score=recall, model_name="DR_Two_Classes")
    print("Completed")